In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import pandas_market_calendars as mcal

## Payoff Function

In [2]:
def payoff(paths, K, Barrier, ratio):
    Nsim = paths.shape[0]
    final_prices = paths[:, :, -1] #S_T
    above_strike = np.all(final_prices >= K, axis=1)
    barrier_event = np.any(paths.min(axis=2) <= Barrier, axis=1)

    # No barrier event or (barrier event and above strike)
    cash_payout = np.where(np.logical_or(np.logical_not(barrier_event), np.logical_and(barrier_event,  above_strike)), 1105, 0)

    # barrier event and below strike
    percent_change = final_prices / K # Based on final strike price
    min_loc = np.argmin(percent_change, axis=1) # Finding the worst performing underlying
    j = np.indices(min_loc.shape)
    payouts = ratio[min_loc] * final_prices[j, min_loc] + 105 # certificate to stock ratio * final price of the stock + coupon payment
    stock_payout = np.where(np.logical_and(barrier_event,  np.logical_not(above_strike)), payouts, 0)

    return np.sum(np.concatenate((cash_payout.reshape(1, Nsim), stock_payout.reshape(1, Nsim))), axis=0)

Expected Payoff: $ \hat{f}(S, t) = \frac{1}{n} \sum^n_{i=1} e^{-r(T-t)} \chi(S^{(i)})$

In [3]:
def expected_payoff(payoffs, r, T, t):
    return np.mean(np.exp(-r*(T-t)) * payoffs)

## Plotting

In [4]:
def plot_simulations(price_hist, Nsim, sim_paths, stock=0):
    """plots simulations for chosen stock

    Args:
        price_hist (df): df of historical prices
        sim_paths (list): simulated price paths
        stock (int, optional): index of stock. Defaults to 0.
    """
    # df of historical prices
    price_hist = price_hist.to_numpy()[:, [0]].reshape(-1)
    price_hist = np.vstack([price_hist]*Nsim)
    price_hist = pd.DataFrame(price_hist).transpose()

    sim_paths = pd.DataFrame(sim_paths[:, stock, :]).transpose()
    sim_paths = pd.concat((price_hist, sim_paths))
    sim_paths = sim_paths.reset_index(drop=True)
    fig = px.line(sim_paths)
    return fig

# GBM Simlulations


## Standard MC

In [5]:
def multi_asset_GBM(S0, r, Sigma, dt, m, p):
    """GBM simulation of multiple asset paths

    Args:
        S0 (vector of start prices): vector of starting prices
        r (int): risk free interest rate
        Sigma (matrix): covariance matrix
        dt (float): delta between each time step
        m (int): number of days simulated
        p (int): number of assets
    """

    S = np.zeros(shape=(m, p))
    S[0] = S0

    z = np.random.multivariate_normal(mean=np.ones(p), cov=np.identity(p), size=m)
    for step in range(1, m):
        S[step] = S[step-1] * np.exp(( r * dt * np.ones(p) ) # Rate discount
             - ( 0.5 * dt * np.diagonal(Sigma) ) # Volatility
             + ( dt * np.matmul(np.linalg.cholesky(Sigma), z[step-1]) )) # Weiner process
    return np.transpose(S)

## Antithetic Variate

In [14]:
def multi_asset_GBM_av(S0, r, Sigma, dt, m, p):
    """GBM simulation of multiple asset paths

    Args:
        S0 (vector of start prices): vector of starting prices
        r (int): risk free interest rate
        Sigma (matrix): covariance matrix
        dt (float): delta between each time step
        m (int): number of days simulated
        p (int): number of assets
    """

    S = np.zeros(shape=(m, p))
    Stilde = np.zeros(shape=(m, p))
    S[0] = S0
    Stilde[0] = S0

    z = np.random.multivariate_normal(mean=np.ones(p), cov=np.identity(p), size=m)
    for step in range(1, m):
        S[step] = S[step-1] * np.exp(( r * dt * np.ones(p) ) # Rate discount
             - ( 0.5 * dt * np.diagonal(Sigma) ) # Volatility
             + ( dt * np.matmul(np.linalg.cholesky(Sigma), z[step-1]) )) # Weiner process

        Stilde[step] = S[step-1] * np.exp(( r * dt * np.ones(p) ) # Rate discount
             - ( 0.5 * dt * np.diagonal(Sigma) ) # Volatility
             + ( dt * np.matmul(np.linalg.cholesky(Sigma), -z[step-1]) )) # Weiner process
    return np.transpose(S), np.transpose(Stilde)

## Helper Functions

In [12]:
# function to get v and Sigma from within a 1 year window
def get_simulation_params(asset_hist, t, lifetime, dt):
    prices = asset_hist.loc[:lifetime[t]].iloc[-253:-1]
    log_prices = prices.apply(lambda x: np.log(x))
    log_returns = log_prices.apply(lambda x: np.diff(x))
    S0 = prices.iloc[-1].to_numpy()
    Sigma = (log_returns.cov()/dt).to_numpy()
    return S0, Sigma

def n_path_sim(asset_hist, Nsim, t, lifetime, dt, p, r, remaining_steps, sim_func, var_reduction=None):
    paths = None

    if var_reduction == None:
        paths = np.zeros(shape=(Nsim, p, remaining_steps))
        for i in range(Nsim):
            S0, Sigma = get_simulation_params(asset_hist, t, lifetime, dt)
            paths[i] = sim_func(S0, r, Sigma, dt, remaining_steps, p)

    elif var_reduction == "av":
        paths = np.zeros(shape=(2 * Nsim, p, remaining_steps))
        for i in range(Nsim):
            S0, Sigma = get_simulation_params(asset_hist, t, lifetime, dt)
            paths[i], paths[Nsim + i] = sim_func(S0, r, Sigma, dt, remaining_steps, p)
 
    return paths
        
        

# Run Simulations

In [8]:
df = pd.read_csv("assets.csv", index_col='Date', parse_dates=True)
df.head()

,CVX,UNH,XOM
Date,,,
2020-05-26,93.300003,294.890015,45.910000
2020-05-27,93.900002,303.769989,46.240002
2020-05-28,90.870003,303.970001,45.040001
2020-05-29,91.699997,304.850006,45.470001
2020-06-01,92.790001,305.929993,46.279999


In [9]:
# Create a calendar
nyse = mcal.get_calendar('NYSE')

# set vars for product lifetime
initial_fixing = pd.to_datetime("2021-05-25")
maturity = pd.to_datetime("2022-11-29")
lifetime = nyse.schedule(start_date=initial_fixing, end_date=maturity).index

# gathered from product description
K = np.array([103.87, 413.05, 58.26])
Barrier = np.array([62.322, 247.830, 34.956])
ratio = np.array([9.6274, 2.4210, 17.1644])

m = len(lifetime)  # no of days product is active for 
T = 1 # Scale from [0,1]
dt = T/m  # daily increment

In [15]:
# ROLLING WINDOW OVER ENTIRE LIFETIME
# uses newest asset prices on current day to get expected payoff for that day
# make sure to run previous cell before running this one
expected_payoffs = np.zeros(len(lifetime))
for t in range( m ):
    paths = n_path_sim(asset_hist=df,
                    Nsim=100,
                    t=t,
                    lifetime=lifetime,
                    dt=dt,
                    p=3,
                    r=0.04,
                    remaining_steps=m-t,
                    sim_func=multi_asset_GBM_av,
                    var_reduction="av"
                    )

    expected_payoffs[t] = expected_payoff(payoff(paths, K, Barrier, ratio), r=0.04, T=T, t=t*dt)

In [16]:
px.line(expected_payoffs)